## 前言

* 黑色星期五购物数据集；

* 关于数据集的描述可以参考[黑五购物数据集](https://www.heywhale.com/mw/dataset/5c6b6ece336a0d002c18b49c)


In [1]:
# 导入需要的库
import pandas as pd
from pyecharts.charts import *
from pyecharts import options as opts
from pyecharts.commons.utils import JsCode

In [2]:
# 使用pandas读取数据
df = pd.read_csv('/home/mw/input/blackfri2862/BlackFriday.csv')
df.head()

,User_ID,Product_ID,Gender,Age,Occupation,City_Category,Stay_In_Current_City_Years,Marital_Status,Product_Category_1,Product_Category_2,Product_Category_3,Purchase
0,1000001,P00069042,F,0-17,10,A,2,0,3,NaN,NaN,8370
1,1000001,P00248942,F,0-17,10,A,2,0,1,6.0,14.0,15200
2,1000001,P00087842,F,0-17,10,A,2,0,12,NaN,NaN,1422
3,1000001,P00085442,F,0-17,10,A,2,0,12,14.0,NaN,1057
4,1000002,P00285442,M,55+,16,C,4+,0,8,NaN,NaN,7969


## 性别

* 男性消费者占比71.72%；

In [3]:
# 按照性别分组，对userid做去重计数，计算男、女消费者分别多少用户
df_t = df.groupby(['Gender']).agg({'User_ID':pd.Series.nunique}).reset_index()
# 通过list对数据进行封装
data_pair = [(row['Gender'], row['User_ID']) for _, row in df_t.iterrows()]

# 新建一个饼图
chart = Pie(
    init_opts=opts.InitOpts(
        theme='light',  # 主题
        width='500px',  # 画布大小
        height='500px'
    )
)

chart.add(
    '',
    data_pair,
    radius=["30%", "45%"],  # 半径范围，内径和外径
    label_opts=opts.LabelOpts(formatter="{b}: {d}%"  # 标签设置，{d}表示显示百分比
    )
)
chart.set_global_opts(
    title_opts=opts.TitleOpts(title="性别占比"),
    legend_opts=opts.LegendOpts(
        is_show=True,
        pos_left='left',
        pos_top='5%',
    ),
)

chart.render_notebook()

## 年龄

* 主力消费群体集中于26-35岁年龄段的人群；

In [4]:
# 按年龄段分组，去重计数
df_t = df.groupby(['Age']).agg({'User_ID':pd.Series.nunique}).reset_index()
data_pair = [(row['Age'], row['User_ID']) for _, row in df_t.iterrows()]

# 新建一个饼图
chart = Pie(
    init_opts=opts.InitOpts(
        theme='light',  # 主题
        width='500px',  # 画布大小
        height='500px'
    )
)

chart.add(
    '',
    data_pair,
    radius=["30%", "45%"],  # 半径范围，内径和外径
    label_opts=opts.LabelOpts(formatter="{b}: {d}%"  # 标签设置，{d}表示显示百分比
    )
)
chart.set_global_opts(
    title_opts=opts.TitleOpts(title="年龄占比"),  # 标题设置
    # 图例设置
    legend_opts=opts.LegendOpts(
        is_show=True,  # 是否显示图例
        pos_left='left',  # 图例距离左边边缘距离
        pos_top='5%',  # 图例距离上边边缘距离
    ),
)

chart.render_notebook()

## 年龄X性别

* 结合性别来看，26-35之间的男性消费群体最大；

In [5]:
# 根据年龄和性别进行分组，去重计数
df_t = df.groupby(['Age', 'Gender']).agg({'User_ID':pd.Series.nunique}).reset_index()

data_x = df_t['Age'][df_t.Gender=='F'].tolist()   # 将pd.series转为list
data_y_f = df_t['User_ID'][df_t.Gender=='F'].tolist()  # 将pd.series转为list
data_y_m = df_t['User_ID'][df_t.Gender=='M'].tolist()  # 将pd.series转为list

# 新建一个Bar
chart = Bar(
    init_opts=opts.InitOpts(
        theme='light',
        width='980px',
        height='500px'
    )
)

# 添加x轴数据
chart.add_xaxis(
    data_x
    )

# 添加y轴数据
chart.add_yaxis(
    'Female',  # 系列名称
    data_y_f,  # 数据
)

# 添加y轴数据，第二个系列
chart.add_yaxis(
    'Male',  # 系列名称
    data_y_m
)

chart.set_global_opts(
    # 标题设置
    title_opts=opts.TitleOpts(
        title="各年龄段用户数量",
        pos_left='center'  # 标题展示位置
        ),
    # 图例设置
    legend_opts=opts.LegendOpts(
        is_show=True,
        pos_left='75%',
        pos_top='1%',
        orient='horizontal',  # 图例水平布局
    ),
)

chart.render_notebook()

## 各年龄段性别占比

* 整体男女比例中，女性28.28%，但是在0-17岁群体中，女性群体占比达到了35%；

In [6]:
# 根据年龄和性别进行分组，去重计数
df_t = df.groupby(['Age', 'Gender']).agg(
    {'User_ID': pd.Series.nunique}).reset_index()

data_x = df_t['Age'][df_t.Gender == 'F'].tolist()  # 将pd.series转为list
data_y_f = df_t['User_ID'][df_t.Gender == 'F'].tolist()  # 将pd.series转为list
data_y_m = df_t['User_ID'][df_t.Gender == 'M'].tolist()  # 将pd.series转为list

# 转为百分比的数据
# Bar图没办法直接获取百分比，自己手动算一下
data_y_f_p, data_y_m_p = [], []
for f, m in zip(data_y_f, data_y_m):
    fp = round(f * 100 / (f + m), 1)
    mp = 100 - fp
    data_y_f_p.append(fp)
    data_y_m_p.append(mp)

# 新建一个Bar
chart = Bar(
    init_opts=opts.InitOpts(
        theme='light',
        width='980px',
        height='500px'
    )
)

# 添加X轴数据
chart.add_xaxis(
    data_x
)

# 添加第一个系列的数据
chart.add_yaxis(
    'Female',
    data_y_f_p,
    stack='aaa',  # stack参数一致的时候 产生堆叠
    label_opts=opts.LabelOpts(position='insideRight', formatter="{c}%")  # 标签设置
)

chart.add_yaxis(
    'Male',
    data_y_m_p,
    stack='aaa',  # 与上方stack参数保持一致 产生堆叠
    label_opts=opts.LabelOpts(position='insideLeft', formatter="{c}%")
)

chart.set_global_opts(
    # 标题设置
    title_opts=opts.TitleOpts(
        title="年龄X性别占比",
        pos_left='center'
    ),
    # 图例设置
    legend_opts=opts.LegendOpts(
        is_show=True,
        pos_left='75%',
        pos_top='1%',
        orient='horizontal',
    ),
)

# xy轴翻转
chart.reversal_axis()
chart.render_notebook()


## 购买力（年龄/性别）

* 在不同年龄段，男性的购买力都要略高于女性；

In [7]:

# 渐变配色设置代码
color_js_2 = """
            new echarts.graphic.LinearGradient(
                                0,
                                1,
                                0,
                                0,
                                [{offset: 0, color: '#00BFFF'},
                                 {offset: 1, color: '#DA70D6'}],
                                false)
           """

color_js_1 = """
            new echarts.graphic.LinearGradient(
                                0,
                                1,
                                0,
                                0,
                                [{offset: 0, color: '#FF4500'},
                                 {offset: 1, color: '#FFE4E1'}],
                                false)
           """

In [10]:
# 分组求平均值
df_t = df.groupby(['User_ID', 'Age', 'Gender']).agg({'Purchase':'mean'}).reset_index()
# 取整
df_t['Purchase'] = df_t['Purchase'].astype('int')

# 新建一些空的list存放数据
data_x = []
data_y_f, data_y_m = [], []
age_pieces = df_t['Age'].unique()
for a in age_pieces:
    for g in ['F', 'M']:
        if g == 'F':
            data_y_f.append(df_t['Purchase'][(df_t.Age==a) & (df_t.Gender==g)].tolist())
        else:
            data_y_m.append(df_t['Purchase'][(df_t.Age==a) & (df_t.Gender==g)].tolist())
            data_x.append(a)

# 箱形图
chart = Boxplot(
    init_opts=opts.InitOpts(
        theme='light',
        width='980px',
        height='500px'
    )
)
# 箱型图数据处理
# 计算最大值，最小值，中位数等等
data_y_f = chart.prepare_data(data_y_f)
data_y_m = chart.prepare_data(data_y_m)

# 添加x轴数据
chart.add_xaxis(
    data_x)

# 添加y轴数据
chart.add_yaxis(
    'Female',
    data_y_f,
    itemstyle_opts={  # 图元样式
        'borderColor': JsCode(color_js_1),
        'color': JsCode(color_js_1)
    },
)

# 添加y轴数据
chart.add_yaxis(
    'Male',
    data_y_m,
    itemstyle_opts={
        'borderColor': JsCode(color_js_2),
        'color': JsCode(color_js_2)
    },
)

chart.set_global_opts(
    # 标题设置
    title_opts=opts.TitleOpts(
        title="各年龄段消费水平",
        pos_left='center'
        ),
    # 坐标轴名称设置
    yaxis_opts = opts.AxisOpts(name='消费'),
    # 图例设置
    legend_opts=opts.LegendOpts(
        is_show=True,
        pos_left='75%',
        pos_top='1%',
        orient='horizontal',
        legend_icon='circle'  # 设置图例图形为实心圆
    ),
)

chart.render_notebook()

## Task 今日份小作业

* 通过饼图展示消费者中，已婚与未婚的**比例**（标签需展示百分比）；

![Image Name](https://cdn.kesci.com/upload/image/refzv4dh73.png?imageView2/0/w/960/h/960)


* 通过饼图展示A/B/C三类城市分别**消费了多少美元**（标签需展示数值）；

![Image Name](https://cdn.kesci.com/upload/image/refzulra18.png?imageView2/0/w/960/h/960)


* 通过柱状图展示，在当前城市停留时间（Stay_In_Current_City_Years）、性别（Gender）的用户对应的**平均消费**多少；

![Image Name](https://cdn.kesci.com/upload/image/refzxuiskf.png?imageView2/0/w/960/h/960)


